# Project: Train a Deep Learning Model to Classify Images (Cats vs Dogs)

## Step 1: Environment Setup

In [3]:
!pip install torch torchvision matplotlib

  Using cached torch-2.7.1-cp311-none-macosx_11_0_arm64.whl.metadata (29 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 10.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchvision] [torchvision]


## Step 2: Load Dataset

we will use a small version of the Dogs vs Cats dataset from torchvision.datasets

In [5]:
pip show torch

Name: torch
Version: 2.7.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchvision
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Transform: Resize and normalize images
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [10]:
# Load CIFAR-10 (we'll use it to simulate cats vs dogs)
# Class 3 = Cat, Class 5 = Dog

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

# Filter only cats and dogs
def filter_animals(dataset):
    indices = [i for i, (_, label) in enumerate(dataset) if label in [3,5]]
    return torch.utils.data.Subset(dataset, indices)

trainloader = DataLoader(filter_animals(trainset), batch_size=32, shuffle=True)
testloader = DataLoader(filter_animals(testset), batch_size=32, shuffle=False)

100%|████████████████████████████████████████| 170M/170M [00:08<00:00, 20.6MB/s]


## Step 3: Define the CNN Model

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # [B, 16, 64, 64]
        x = self.pool(F.relu(self.conv2(x))) # [B, 32, 32, 32]
        x = x.view(-1, 32*32*32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

model = CNN()